In [1]:
# -*- coding: utf-8 -*-
# Normal Python packages
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
import numpy as np
from glob import glob
import os

# Plotting packages
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
from shapely.geometry import box, LineString, Point
import cmocean as cm

# Add the path to the Tatsu's swot library
import sys
sys.path.append('../src/')
import swot_utils
import data_loaders
import download_swaths


# Turn off warnings
import warnings
warnings.filterwarnings("ignore")

# Load the swath/pass combos you want

Assuming you already have everything downloaded..

In [2]:
# Specify the path to the subsetted data
L3_cali_path = f"../SWOT_L3/Unsmoothed_cali"

# Define domain
# Rough West of California domain
cali_sw_corner = [-125,38.5] # full area [-134.0994348261152,27.77869536057762]
cali_ne_corner = [-123,42.5] # full area [-115.2951365210726,45.43712363060855]
lat_lims = [cali_sw_corner[1],cali_ne_corner[1]]

# Define mission phase (1-day repeat vs science) and 
# cycles we are interested in
# Use sph_calval_swath for the 1-day repeats
path_to_sph_file="../orbit_data/sph_calval_swath.zip"
# Cycles 474 - 578 are from the 1-day repeat 
cycles = [str(c_num).zfill(3) for c_num in range(474,579)] # 579

pass_IDs_list = download_swaths.find_swaths(cali_sw_corner, cali_ne_corner,
                                           path_to_sph_file=path_to_sph_file)

# Be sure to set subset as True when taking a subset of the data defined in notebook 2!

cycle_data = {}
for cycle in cycles:
    cycle_data[cycle] = data_loaders.load_cycle(L3_cali_path,fields=["time","ssha","ssha_unedited","ssha_noiseless","sigma0"],
                                                cycle=cycle,pass_ids=pass_IDs_list,subset=True,lats=lat_lims
                                               )


Loading SWOT_L3_LR_SSH_Unsmoothed_474_013_20230329T081622_20230329T090619_v1.0.2_cali.nc
Loading SWOT_L3_LR_SSH_Unsmoothed_478_013_20230402T073744_20230402T082850_v1.0.2_cali.nc
Loading SWOT_L3_LR_SSH_Unsmoothed_479_013_20230403T072822_20230403T081927_v1.0.2_cali.nc
Loading SWOT_L3_LR_SSH_Unsmoothed_480_013_20230404T071859_20230404T081005_v1.0.2_cali.nc
Loading SWOT_L3_LR_SSH_Unsmoothed_481_013_20230405T070937_20230405T080043_v1.0.2_cali.nc
Loading SWOT_L3_LR_SSH_Unsmoothed_482_013_20230406T070015_20230406T075121_v1.0.2_cali.nc
Loading SWOT_L3_LR_SSH_Unsmoothed_483_013_20230407T065053_20230407T074159_v1.0.2_cali.nc
Loading SWOT_L3_LR_SSH_Unsmoothed_484_013_20230408T064131_20230408T073237_v1.0.2_cali.nc
Loading SWOT_L3_LR_SSH_Unsmoothed_485_013_20230409T063210_20230409T072315_v1.0.2_cali.nc
Loading SWOT_L3_LR_SSH_Unsmoothed_486_013_20230410T062248_20230410T071353_v1.0.2_cali.nc
Loading SWOT_L3_LR_SSH_Unsmoothed_487_013_20230411T061326_20230411T070432_v1.0.2_cali.nc
Loading SWOT_L3_LR_SS

# GCM filters
# sci-py convolutional/gaussian filters

In [3]:
import gcm_filters 

In [25]:
ds = cycle_data['474'][0]
ssha = ds['ssha']
ssha

<xarray.DataArray 'ssha' (num_lines: 1914, num_pixels: 519)> Size: 8MB
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], shape=(1914, 519))
Coordinates:
    latitude   (num_lines, num_pixels) float64 8MB 38.63 38.63 ... 42.36 42.36
    longitude  (num_lines, num_pixels) float64 8MB 234.6 234.6 ... 237.4 237.4
Dimensions without coordinates: num_lines, num_pixels
Attributes:
    comment:        Height of the sea surface anomaly with all corrections ap...
    long_name:      sea surface height anomaly
    standard_name:  sea_surface_height_above_reference_ellipsoid
    units:          m

In [46]:
dy0 = 1852*60*np.abs(ds['latitude'][2].data - ds['latitude'][1].data)
# ** note: dy0 is a constant over the globe ** 
dx0 = 1852*60*np.cos(np.deg2rad(ds['latitude'].data))*(ds['longitude'][2].data - ds['longitude'][1].data) 
area = dy0*np.tile(dx0, (len(ds['longitude'].data),1))
area = np.transpose(area)
dA = xr.DataArray(
    data=area,
    dims=["latitudes", "longitude"],
    coords=dict(
    longitude=(["longitude"], ds['longitude'].data), latitude=(["latitude"], ds['latitude'].data),),
)

ValueError: Variable 'longitude': Could not convert tuple of form (dims, data[, attrs, encoding]): (['longitude'], array([[234.64149, 234.6443 , 234.64711, ..., 236.08918, 236.09198,
        236.09479],
       [234.64207, 234.64487, 234.64768, ..., 236.08979, 236.09259,
        236.0954 ],
       [234.64267, 234.64548, 234.64829, ..., 236.09041, 236.09321,
        236.09602],
       ...,
       [235.84596, 235.8489 , 235.85185, ..., 237.36637, 237.36932,
        237.37226],
       [235.84718, 235.85012, 235.85306, ..., 237.36428, 237.36722,
        237.37016],
       [235.84604, 235.84899, 235.85194, ..., 237.36592, 237.36886,
        237.37181]], shape=(1914, 519))) to Variable.

In [45]:
dx0

array([[52.08471271, 52.95316052, 52.95352979, ..., 54.01447487,
        54.01484807, 54.01522873],
       [52.08317991, 52.95160219, 52.95197147, ..., 54.01292234,
        54.01329555, 54.01367622],
       [52.08166157, 52.95005856, 52.95042786, ..., 54.01134733,
        54.01172056, 54.01210125],
       ...,
       [49.05184485, 49.86980828, 49.87024095, ..., 50.91322047,
        50.91365791, 50.91409534],
       [49.05029222, 49.86822177, 49.86865446, ..., 50.9110899 ,
        50.91152736, 50.91196482],
       [49.04840061, 49.86630667, 49.86673137, ..., 50.90956684,
        50.91000432, 50.91044179]], shape=(1914, 519))

In [33]:
print(data_.shape)

NameError: name 'data_' is not defined

In [30]:
# -- wet mask -- 
# -- land = 0, water = 1 -- 
wetMask = xr.where(np.isnan(ssha), 0, 1)
wetMask

<xarray.DataArray 'ssha' (num_lines: 1914, num_pixels: 519)> Size: 8MB
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(1914, 519))
Coordinates:
    latitude   (num_lines, num_pixels) float64 8MB 38.63 38.63 ... 42.36 42.36
    longitude  (num_lines, num_pixels) float64 8MB 234.6 234.6 ... 237.4 237.4
Dimensions without coordinates: num_lines, num_pixels

In [20]:
# -- choose a filtering scale -- 
filter_scale = 3
dx_min = 1
# -- initialze filter object for two filter types -- 
specs = {
    'filter_scale': filter_scale,
    'dx_min': dx_min,
    'grid_type': gcm_filters.GridType.REGULAR_WITH_LAND_AREA_WEIGHTED,
    'grid_vars': {'area': dA, 'wet_mask': wetMask}
}
# GAUSSIAN 
filter_simple_fixed_factor_G = gcm_filters.Filter(filter_shape=gcm_filters.FilterShape.GAUSSIAN, **specs)

NameError: name 'dA' is not defined